Notebook for inputting the final state momenta back into events after reconstruction (from baler), requires a npz of px1,py1,pz1,e1,px2,py2,pz2,e2.

Requries pyhepmc.

In [ ]:
import numpy as np
import pyhepmc
from pyhepmc.io import ReaderAsciiHepMC2, WriterAsciiHepMC2
from pyhepmc import GenEvent

def read_npz_and_split_data(input_path):
    data = np.load(input_path)["data"]
    return np.vstack((np.expand_dims(data[:, :4], axis = 0), np.expand_dims(data[:, 4:], axis = 0),))

def replace_final_state_4_momenta(p1_data, p2_data, input_file, output_file):
    count = 0
    with ReaderAsciiHepMC2(input_file) as reader, \
         WriterAsciiHepMC2(output_file) as writer:

        evt = GenEvent()
        while not reader.failed():
            reader.read_event(evt)
            if reader.failed():
                break
            e_pair = []
            for p in evt.particles:
                if p.pid == 23:
                    ev = p.end_vertex
                    for dau in ev.particles_out:
                        if dau.status == 1 and abs(dau.pid) == 11:
                            e_pair.append(dau)
            e_pair[0].momentum = pyhepmc.FourVector(p1_data[count,0], p1_data[count,1], p1_data[count, 2], p1_data[count, 3]) 
            e_pair[1].momentum = pyhepmc.FourVector(p2_data[count,0], p2_data[count,1], p2_data[count, 2], p2_data[count, 3])         

            count += 1                    
            writer.write(evt)

    
def replace_4_momenta(npz_file, input_files, output_files):
    assert len(input_files) == len(output_files)
    num_files = len(input_files)
    num_events = 10_000
    num_dims = 4
    num_parts = 2
    momenta_data = read_npz_and_split_data(npz_file)
    momenta_data = momenta_data.reshape((num_parts , num_files, num_events, num_dims))
    for (input, output, p1_data, p2_data) in zip(input_files, output_files, momenta_data[0], momenta_data[1]):
        replace_final_state_4_momenta(p1_data, p2_data, input, output)



In [29]:
replace_4_momenta("../processedData/decompressed100-000.npz",
                   [
    f"../DATA/HEPMC.43646139._0000{str(i).zfill(2)}.hepmc" for i in range(1, 11)
],
[ f"../processedData/reinputed_final_state{str(i).zfill(2)}.hepmc" for i in range (1, 11)]
)